# Summer Irrigated Cropping Area (SICA) Processing Pipeline Infra Deisgn

As the SICA design, the whole processing is:

<div id='svgWrapper'>
    <img src='sica_overall.svg'/>
</div>

There are two main steps:
1. Summer period datasets to Max NDVI result dataset
2. Load whole Max NDVI result datasets and mask, and generate crop area polygon

The step 1 is close to the `DEA odc-stats` product processing pattern: `Statistician is a framework of tools for generating statistical summaries of large collections of EO data managed in an ODC instance.` Each processing unit will only focus on a single region.
The step 2 is loading data from different data sources, and processing. We should custom an application. We would only have one processing unit.

What is more, the step 1 should guarantee all processing unit are job done, then start the step 2. Ideally, we can use Argo DAG or Airflow DAG to make sure this happen. The detail AWS infra would be:

<div id='svgWrapper'>
    <img src='SICA.drawio.svg'/>
</div>

###  Argo/Airflow Step 1

It will start several EC2 instances (by DEA EKS API) to run odc-stats Max NDVI plugin (via `odc-stats Docker image`), to create summer period Max NDVI result to each region (e.g. one region example: https://explorer.prod.dea.ga.gov.au/product/ga_ls5t_ard_3/regions/099080). The Max NDVI result will be archived in DEA product AWS S3 bucket.

### Argo/Airflow Step 2
It will start a `SICA custom application` with `SICA custom Docker image` in a huge EC2 instance. This EC2 should have capacity to load all Max NDVI result which belong to same summer period. Because we are using the Argo/Airflow DAG to confirm the step 2 will be run after step 1, so there is `NO` need to run sanity check to confirm all regions have the Max NDVI result. 

### Conclusion

If we plan to add the SICA product to DEA environment, we have to do following tasks:

1. a Max NDVI plugin to odc-stats repo
2. custom SICA application can load Max NDVI result, masking by other datasets (e.g. rainfall), and generate the polgyons based on thresholds.
3. SICA Bitbucket/Github repo + SICA docker image
4. new Argo/Airflow DAG
5. AWS resource access control (e.g. interface to load rainfall dataset, new SICA resource group)

### TBD / random ideas from Sai

1. the step 2 can change to `each region create its own polgyon, and merge them at last step`. But the current desgin is more like the common solution.
2. the step 2 application is more like a huge Jupyter Notebook. We can just create a notebook to finish the step 2 job. But the SICA processing is not fully automatic .